In [2]:
from glob import glob
from functions import read_file, build_pipeline
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, GridSearchCV

## Topic Inference

In [3]:
cates_dir = glob('data/20news-18828/*')

topics_path = {c.split('/')[-1]: glob(f'{c}/*') for c in cates_dir}

print("Total topics (classes): ", len(topics_path)) 

df = DataFrame([(k, v) for k, v in topics_path.items()], columns=['topics', 'files'])

df = df.explode('files')

print("Total files: ", len(df))

df['text'] = df['files'].apply(read_file)

df.value_counts('topics')

Total topics (classes):  20
Total files:  18828


topics
rec.sport.hockey            999
soc.religion.christian      997
rec.motorcycles             994
rec.sport.baseball          994
sci.crypt                   991
rec.autos                   990
sci.med                     990
sci.space                   987
comp.os.ms-windows.misc     985
comp.sys.ibm.pc.hardware    982
sci.electronics             981
comp.windows.x              980
comp.graphics               973
misc.forsale                972
comp.sys.mac.hardware       961
talk.politics.mideast       940
talk.politics.guns          910
alt.atheism                 799
talk.politics.misc          775
talk.religion.misc          628
Name: count, dtype: int64

In [4]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['topics'], train_size=0.7, random_state=42)

print("Train: ", len(x_train))
print("Test: ", len(x_test))

Train:  13179
Test:  5649


In [5]:
cnt_pipeline = build_pipeline('count').fit(x_train)
tfidf_pipeline = build_pipeline('tfidf').fit(x_train)

In [6]:
cv = StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=1/7) # Validation is around 10% of original dataset

In [7]:
X_train_transformed = tfidf_pipeline.transform(x_train)

## Regresión Logística 

In [8]:
# Entrenar un clasificador
estimator = LogisticRegression(n_jobs=-1, random_state=42, 
                         class_weight='balanced', solver='saga',
                         max_iter=1000, penalty='l2')

param_grid = {
    'C': [1, 10],
}


grid_search_best_estimator = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    return_train_score=False,
    refit=True
).fit(X_train_transformed, y_train)

grid_search_best_estimator.cv_results_

{'mean_fit_time': array([ 6.92132976, 13.00456645]),
 'std_fit_time': array([0.47966842, 1.6608313 ]),
 'mean_score_time': array([0.04260573, 0.01889541]),
 'std_score_time': array([0.01208735, 0.0091829 ]),
 'param_C': masked_array(data=[1, 10],
              mask=[False, False],
        fill_value=999999),
 'params': [{'C': 1}, {'C': 10}],
 'split0_test_score': array([0.88469509, 0.90540793]),
 'split1_test_score': array([0.88197685, 0.9039375 ]),
 'split2_test_score': array([0.87683065, 0.89847258]),
 'split3_test_score': array([0.89030482, 0.90925725]),
 'split4_test_score': array([0.86866562, 0.89129165]),
 'split5_test_score': array([0.87007643, 0.89661617]),
 'split6_test_score': array([0.88704218, 0.90643806]),
 'split7_test_score': array([0.90051016, 0.91844248]),
 'split8_test_score': array([0.87857204, 0.9053864 ]),
 'split9_test_score': array([0.88901983, 0.90796304]),
 'mean_test_score': array([0.88276937, 0.9043213 ]),
 'std_test_score': array([0.00919833, 0.00712268]),
 

In [9]:
# Evaluar el modelo
X_test_transformed = tfidf_pipeline.transform(x_test)
y_pred = grid_search_best_estimator.predict(X_test_transformed)
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.92      0.88      0.90       249
           comp.graphics       0.81      0.88      0.85       279
 comp.os.ms-windows.misc       0.88      0.82      0.85       280
comp.sys.ibm.pc.hardware       0.81      0.83      0.82       316
   comp.sys.mac.hardware       0.90      0.89      0.90       283
          comp.windows.x       0.88      0.89      0.88       292
            misc.forsale       0.86      0.89      0.87       298
               rec.autos       0.93      0.91      0.92       305
         rec.motorcycles       0.97      0.96      0.96       276
      rec.sport.baseball       0.96      0.96      0.96       302
        rec.sport.hockey       0.95      0.98      0.97       301
               sci.crypt       0.98      0.93      0.95       301
         sci.electronics       0.89      0.91      0.90       292
                 sci.med       0.94      0.96      0.95       320
         

## Naive Bayes

In [18]:
estimator = MultinomialNB()

param_grid = {
    'alpha': [0.01, 0.1, 1],
}


grid_search_best_estimator = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    return_train_score=False,
    refit=True
).fit(X_train_transformed, y_train)

grid_search_best_estimator.cv_results_

{'mean_fit_time': array([0.12618616, 0.13382003, 0.10436502]),
 'std_fit_time': array([0.02513395, 0.03558606, 0.00985353]),
 'mean_score_time': array([0.02255607, 0.02746015, 0.01695735]),
 'std_score_time': array([0.00228209, 0.00913215, 0.00584362]),
 'param_alpha': masked_array(data=[0.01, 0.1, 1.0],
              mask=[False, False, False],
        fill_value=1e+20),
 'params': [{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}],
 'split0_test_score': array([0.89648046, 0.88887367, 0.85472444]),
 'split1_test_score': array([0.89135108, 0.88950543, 0.85422328]),
 'split2_test_score': array([0.8897076 , 0.88604202, 0.84697417]),
 'split3_test_score': array([0.90468242, 0.90117251, 0.86388099]),
 'split4_test_score': array([0.8734912 , 0.86290117, 0.83096433]),
 'split5_test_score': array([0.88137618, 0.87806768, 0.85126581]),
 'split6_test_score': array([0.88574744, 0.8835521 , 0.84466198]),
 'split7_test_score': array([0.89753053, 0.89568674, 0.86147626]),
 'split8_test_score': array([

In [20]:
# Evaluar el modelo
y_pred = grid_search_best_estimator.predict(X_test_transformed)
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.93      0.90      0.91       249
           comp.graphics       0.74      0.89      0.81       279
 comp.os.ms-windows.misc       0.83      0.77      0.80       280
comp.sys.ibm.pc.hardware       0.80      0.83      0.81       316
   comp.sys.mac.hardware       0.87      0.90      0.89       283
          comp.windows.x       0.85      0.87      0.86       292
            misc.forsale       0.87      0.84      0.85       298
               rec.autos       0.96      0.93      0.95       305
         rec.motorcycles       0.96      0.96      0.96       276
      rec.sport.baseball       0.96      0.96      0.96       302
        rec.sport.hockey       0.97      0.98      0.97       301
               sci.crypt       0.97      0.93      0.95       301
         sci.electronics       0.90      0.87      0.89       292
                 sci.med       0.94      0.94      0.94       320
         